In [2]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [3]:
wavelength = []
intensity =[]
with open("slit9_of_Baldwin_et_al_1991_corrpos_sum_nall.dat","r") as f:
    for line in f.readlines():
        line = line.split(' ')
        wavelength.append(float(line[0]))
        intensity.append(float(line[1]))
        
wavelength = np.array(wavelength)
intensity = np.array(intensity)

In [4]:
plot(wavelength,intensity)

In [5]:
import pyfits
image = np.copy(pyfits.open("emhalpha_flux.fits")[0].data)

In [6]:
imshow(image)

In [7]:
plot(intensity)

In [7]:
def thresh(y,y0):
    """
    Find all intervals above a threshold
    """
    intervals = []
    idx = []
    
    a = None
    high = False
    
    for i,yy in enumerate(y):
        
        if yy >= y0:            
            if high == False:
                a = i
                high = True
        else:
            if high == True:
                intervals.append(y[a:i])
                idx.append(a)
                a = None
            high = False
            
    return np.array(idx), np.array(intervals)
             
        


In [106]:
def perimeter(image, a, w, bins=None):
    
    if bins == None:
        bins = .001 * np.size(image)

    h,b = np.histogram(image,bins=bins)
    b = b[:-1] + .5*np.diff(b)
    dflux = b * h    
    ndflux = dflux/np.sum(dflux)
    
    perimeter_bins = b[np.logical_and( (np.cumsum(ndflux) > a-w) , ((np.cumsum(ndflux) < a+w)) )]
    low = perimeter_bins[0]
    high = perimeter_bins[-1]

    perimeter_x = []
    perimeter_y = []

    for i,row in enumerate(image):
        for j,val in enumerate(row):
            if low <= val <= high:
                perimeter_x.append(i)
                perimeter_y.append(j)
            
    return np.array(perimeter_x), np.array(perimeter_y)

def center(image):
    
    x,y = image.shape
    
    m,cx,cy = 0.,0.,0.
    
    for xx in range(x):
        for yy in range(y):
            val = image[xx][yy]
            cx += val*xx
            cy += val*yy
            m += val
        
    return cx/m, cy/m
        
def peak(image):
    i = np.argmax(image.flatten())
    lx,ly = image.shape
    return i/(ly),i%(ly)

def r_eff(px,py,cx,cy):
    
    r = np.sqrt((px-cx)**2 + (py-cy)**2)/(len(px))
    return np.sum(r)
    

In [107]:
import scipy.ndimage.filters

image2 = scipy.ndimage.filters.gaussian_filter(image,[50,50])

x,y=perimeter(image2,.1,.01,2000)

disp = np.copy(image2)

for xx,yy in zip(x,y):
    disp[xx][yy] = 0

cx,cy = center(image2)
cx2,cy2 = peak(image2)
scatter(cx,cy)
scatter(cx2,cy2)
imshow(disp)



In [46]:
r_eff(x,y,cx,cy)

765.23765838161523

In [108]:
r_eff(x,y,cx2,cy2)

792.96040232888652

In [63]:
np.argmax(image)

1489662

In [29]:
center(image)

(827.88482568063523, 827.88482568063523)

In [54]:
image.shape

(1476, 1766)

In [33]:
1476*.5

738.0

In [34]:
1766*.5

883.0

In [35]:
zip(range(5),range(8))

[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4)]

In [69]:
np.arange(10).reshape(2,5)

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

In [70]:
np.arange(10).reshape(2,5).flatten()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [104]:
a = np.zeros(64)
a=a.reshape(16,4)
a[1,0] = 1

In [77]:
peak(image2)

(836, 1065)

In [105]:
peak(a)

(2, 0)

In [98]:
a

array([[ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.]])